In [1]:
cacf = False

In [2]:
import pyodbc
import pandas as pd
import os
from dotenv import load_dotenv
import datetime
import locale
from fonctions import *
import plotly.express as px
import numpy as np

# Définir la langue française
locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")

if cacf:
    # Load des var env
    load_dotenv()

    server = os.getenv('SERVER')
    database = os.getenv('DATABASE')
    username = os.getenv('USERNAME')
    password = os.getenv('PASSWORD')
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()

    query = "SELECT * FROM dbo.Telephonie;"
    df_tel = pd.read_sql(query, cnxn)

    query = "SELECT * FROM dbo.Eptica;"
    df_ept = pd.read_sql(query, cnxn)
else:
    df_tel = pd.read_csv('tel.csv')
    df_tel = df_tel.drop("Unnamed: 0", axis=1)

In [3]:
df_tel.shape

(23783, 5)

In [4]:
df_tel.columns

Index(['id', 'date_appel', 'entite', 'famille', 'nombre_entrants_corrige'], dtype='object')

## Setup dataset

In [5]:
df_tel['date_appel'] = pd.to_datetime(df_tel['date_appel']) # On encode la column "date_appel" pour l'exploiter en timeseries
df_tel['nombre_entrants_corrige'] = df_tel['nombre_entrants_corrige'].apply(lambda x: x if x > 1 else 1).astype(int) # On transforme les float en Int car un demi appel n'existe pas
df_tel.head()

,id,date_appel,entite,famille,nombre_entrants_corrige
0,12,2019-01-02,Entite 1,F2,165
1,13,2019-01-02,Entite 1,F3,54
2,14,2019-01-02,Entite 1,F4,1
3,15,2019-01-02,Entite 1,F5,99
4,16,2019-01-02,Entite 1,F6,1487


In [6]:
# Nombre d'entités (5)
df_tel['entite'].unique()

array(['Entite 1', 'Entite 2', 'Entite 3', 'Entite 4', 'Entite 5'],
      dtype=object)

In [7]:
# Nombre de famille (7)
df_tel['famille'].unique()

array(['F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F1'], dtype=object)

In [8]:
# On définit nos dates comme index
df_tel = df_tel.set_index(df_tel['date_appel'].rename("date")).drop("date_appel", axis=1)

# Renome la columns appels reçus
df_tel.rename(columns={"nombre_entrants_corrige":"nb_recus"}, inplace=True)

In [9]:
# Ajout des colonnes Jour en Chiffres, Mois, année
df_tel = df_tel.reset_index()
df_tel['jour_int']= df_tel['date'].apply(lambda x: int(x.day))
df_tel['mois']= df_tel['date'].apply(lambda x: str(x.strftime("%B")))
df_tel['annee']= df_tel['date'].apply(lambda x: int(x.year))

In [10]:
# Ajout d'une column jour
df_tel = df_tel.reset_index()
df_tel['jour'] = df_tel['date'].apply(lambda x: str(datetime.datetime(int(x.year), int(x.month), int(x.day)).strftime("%A")))
df_tel = df_tel.set_index('date')

In [11]:
# on garde 2 dernières année
two_last_year = True
if two_last_year:
    df_tel = df_tel.reset_index()
    df_tel = df_tel.loc[(df_tel['date'] > "2020-12-31")] # & (df_tel['date'] < "2023-01-01")]
    df_tel = df_tel.set_index("date")

In [12]:
# Ajout des soldes Hiver & été
df_tel = df_tel.reset_index()
df_tel['solde'] = df_tel['date'].astype(str).apply(lambda x: est_date_soldes(str(x)))
df_tel = df_tel.set_index('date')

In [13]:
# Ajout de saison
df_tel = df_tel.reset_index()
df_tel['saison'] = df_tel['date'].apply(lambda x: trouver_saison(x.date()))
df_tel = df_tel.set_index("date")

In [14]:
# Ajout veille et lendemain jour férié
df_tel = df_tel.reset_index()
df_tel['before_ferie'] = df_tel['date'].astype(str).apply(lambda x: est_veille_ferie(x))
df_tel['after_ferie'] =  df_tel['date'].astype(str).apply(lambda x: est_lendemain_ferie(x))
df_tel = df_tel.set_index("date")

## Tendance Globale

In [15]:
df = df_tel.groupby(pd.Grouper(freq = 'Y')).agg(nb_recus_avg = ('nb_recus' , 'mean')).reset_index()
fig = px.line(df, x="date", y="nb_recus_avg")
fig.show()

## Entités par jour par famille

In [16]:
#
# Création de graphique dynamique de toutes les Entités avec toutes les Familles
#

activer_samedi = False

for entite in df_tel.entite.unique():
    if activer_samedi:
      df = df_tel.loc[(df_tel.entite == entite)].groupby([pd.Grouper(freq = 'd'), "famille", "jour"]).agg(nb_recus = ('nb_recus' , 'sum')).reset_index()
    else:
       df = df_tel.loc[(df_tel.entite == entite) & (df_tel.jour != "Samedi")].groupby([pd.Grouper(freq = 'd'), "famille", "jour"]).agg(nb_recus = ('nb_recus' , 'sum')).reset_index()
    
    fig = px.line(df, x="date", y="nb_recus", color='famille',
              hover_data={
                "date": "|%B %d, %Y",
                "jour" : True,
              },
              title=f"Nombre d'appel pour {entite}")

    fig.update_xaxes(
        dtick="M1",
        tickformat="%b\n%Y",
        ticklabelmode="period")
    fig.show()

## Répartition

In [17]:
import plotly.express as px

fig = px.pie(df_tel, values='nb_recus', names='entite', title="Nombre d'appel reçus par Entité")
fig.show()

In [18]:
fig = px.pie(df_tel, values='nb_recus', names='famille', title="Nombre d'appel reçus par Famille")
fig.show()

In [19]:
fig = px.bar(df_tel.groupby(['famille','entite']).agg(nb_recus = ("nb_recus" , "sum")).reset_index(), x='entite', y='nb_recus', color="famille")
fig.show()

In [20]:

fig = px.bar(df_tel.groupby([pd.Grouper(freq = 'm'), 'entite']).agg(nb_recus = ("nb_recus" , "sum")).reset_index(), x='date', y='nb_recus', color="entite")
fig.show()

In [21]:
fig = px.bar(df_tel.groupby(["entite", "jour", "famille"]).agg(nb_recus = ('nb_recus' , 'sum')).reset_index(), x="jour", y="nb_recus", color="entite", title="Nombre d'appels recus par entite (jour)",
            hover_data={
                "famille" : True,
                "entite" : True,
            })
fig.show()

In [22]:
fig = px.bar(df_tel.groupby(["entite", "mois", "famille"]).agg(nb_recus = ('nb_recus' , 'sum')).reset_index(), x="mois", y="nb_recus", color="entite", title="Nombre d'appels recus par entite (mois)",
            hover_data={
                "famille" : True,
                "entite" : True,
            })
fig.show()

fig = px.line(df_tel.groupby(["entite", "mois"]).agg(nb_recus = ('nb_recus' , 'mean')).reset_index(), x="mois", y="nb_recus", color="entite", title="Moyenne d'appel reçus par les entités par mois",
            hover_data={
                "entite" : True,
            })
fig.show()

In [23]:
fig = px.bar(df_tel.groupby([pd.Grouper(freq="w"), "entite"]).agg(nb_recus = ("nb_recus", "sum")).reset_index(), x="date", y="nb_recus", color="entite", title="Nb Appel par semaine par entité")
fig.show()

In [24]:
df_tel = df_tel.reset_index()
df_tel['vacances'] = df_tel['date'].astype(str).apply(lambda x: is_vacances(x)).astype(int)
df_tel = df_tel.set_index("date")

In [25]:
df_tel.describe() # On cherche le top 25%

,index,id,nb_recus,jour_int,annee,solde,before_ferie,after_ferie,vacances
count,12059.000000,12059.000000,12059.000000,12059.000000,12059.000000,12059.000000,12059.000000,12059.000000,12059.000000
mean,17753.000000,17765.000000,577.789949,15.666473,2021.596650,0.276640,0.023800,0.029853,0.305913
std,3481.277783,3481.277783,706.007235,8.777745,0.604902,0.447355,0.152431,0.170189,0.460812
min,11724.000000,11736.000000,1.000000,1.000000,2021.000000,0.000000,0.000000,0.000000,0.000000
25%,14738.500000,14750.500000,101.000000,8.000000,2021.000000,0.000000,0.000000,0.000000,0.000000
50%,17753.000000,17765.000000,400.000000,16.000000,2022.000000,0.000000,0.000000,0.000000,0.000000
75%,20767.500000,20779.500000,760.000000,23.000000,2022.000000,1.000000,0.000000,0.000000,1.000000
max,23782.000000,23794.000000,4667.000000,31.000000,2023.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
df_tel.loc[df_tel['nb_recus'] > 775]['jour'].value_counts() # Réparatition du nombre de jour dans le top 25%

jour
Lundi       638
Mardi       630
Mercredi    582
Jeudi       529
Vendredi    514
Samedi       42
Name: count, dtype: int64

In [27]:
df_tel.loc[df_tel['nb_recus'] < 86]['jour'].value_counts() # Répartition des plus petits jours

jour
Vendredi    508
Mercredi    503
Mardi       498
Jeudi       488
Lundi       476
Samedi      428
Name: count, dtype: int64

### Férié

#### Lendemain

In [28]:
# Entité x Ferie
fig = px.histogram(df_tel.groupby(["entite", "before_ferie", "after_ferie"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="entite", y="nb_recus",
             color="after_ferie", barmode='group',
             height=400,
             title="Nb Recus par entité lendemain de jour férié")
fig.show()

In [29]:
# Famille x Ferie
fig = px.histogram(df_tel.groupby(["famille", "before_ferie", "after_ferie"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="famille", y="nb_recus",
             color="after_ferie", barmode='group',
             height=400,
             title="Nb Recus par famille lendemain de jour férié")
fig.show()

#### Veille

In [30]:
# Entité x Ferie
fig = px.histogram(df_tel.groupby(["entite", "before_ferie", "after_ferie"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="entite", y="nb_recus",
             color="before_ferie", barmode='group',
             height=400,
             title="Nb Recus par entité veille de jour férié")
fig.show()

In [31]:
# Famille x Ferie
fig = px.histogram(df_tel.groupby(["famille", "before_ferie", "after_ferie"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="famille", y="nb_recus",
             color="before_ferie", barmode='group',
             height=400,
             title="Nb Recus par famille veille de jour férié")
fig.show()

### Saison

In [32]:
# Entité x Saison
fig = px.histogram(df_tel.groupby(["entite", "saison"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="entite", y="nb_recus",
             color='saison', barmode='group',
             height=400,
             title="Nb Recus par entité et par saison")
fig.show()

In [33]:
# Famille x Saison
fig = px.histogram(df_tel.groupby(["famille", "saison"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="famille", y="nb_recus",
             color='saison', barmode='group',
             height=400,
             title="Nb Recus par famille et saison")
fig.show()

### Solde

In [34]:
# Entité x Solde
fig = px.histogram(df_tel.groupby(["entite", "solde"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="entite", y="nb_recus",
             color='solde', barmode='group',
             height=400,
             title="Nb Recus par entité avec et sans solde")
fig.show()

In [35]:
# Famille x Solde
fig = px.histogram(df_tel.groupby(["famille", "solde"]).agg(nb_recus = ("nb_recus", "median")).reset_index(), x="famille", y="nb_recus",
             color='solde', barmode='group',
             height=400,
             title="Nb Recus par famille avec et sans solde")
fig.show()

## Affichage des familles

In [36]:
activer_samedi = False

for famille in df_tel['famille'].unique():
    if activer_samedi:
      df = df_tel.loc[df_tel['famille'] == famille].groupby([pd.Grouper(freq="d"), "entite"]).agg(nb_recus =("nb_recus", "sum")).reset_index()
    else:
       df = df_tel.loc[(df_tel.famille == famille) & (df_tel.jour != "Samedi")].groupby([pd.Grouper(freq="d"), "entite"]).agg(nb_recus =("nb_recus", "sum")).reset_index()

    fig = px.line(df, x="date", y="nb_recus", color="entite",
          hover_data={
            "date": "|%B %d, %Y",
            "entite" : True,
          },
          title=f"Nombre d'appel par Entite pour la famille {famille}")
    fig.show()

## SAV

In [37]:
df = df_tel
df['sav'] = df['famille'].apply(lambda x: 1 if x in ["F7", "F6"] else 0)

In [38]:
activer_samedi = False

if activer_samedi:
  df = df_tel.loc[df_tel['sav'] == 1].groupby([pd.Grouper(freq="d"), "entite"]).agg(nb_recus =("nb_recus", "sum")).reset_index()
else:
   df = df_tel.loc[(df_tel['sav'] == 1) & (df_tel.jour != "Samedi")].groupby([pd.Grouper(freq="d"), "entite"]).agg(nb_recus =("nb_recus", "sum")).reset_index()

fig = px.line(df, x="date", y="nb_recus", color="entite",
          hover_data={
            "date": "|%B %d, %Y",
            "entite" : True,
          },
          title=f"Nombre d'appel par Entité pour le SAV (F6/F7)")
fig.show()

## Correlation

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt

df_corr = pd.get_dummies(df_tel.reset_index().drop(["id", "index", "date"], axis=1), columns=['entite', 'famille', 'mois', 'jour_int', 'annee', 'jour', 'saison'], dtype=int)
corr = df_corr.corr()

corr['nb_recus'].sort_values(ascending=False)

nb_recus           1.000000
sav                0.380113
famille_F7         0.321358
entite_Entite 5    0.280718
entite_Entite 2    0.163107
                     ...   
entite_Entite 3   -0.157785
entite_Entite 1   -0.168424
jour_Samedi       -0.202504
entite_Entite 4   -0.211509
famille_F5        -0.276943
Name: nb_recus, Length: 74, dtype: float64

# Dummy model (Moyenne)

In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder, StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

cat_var = ['famille','jour', 'mois', 'jour_int']
num_var = ['nb_recus', 'vacances']

## Transformers Pipeline
cat_transformer = OneHotEncoder(handle_unknown='ignore')
num_transformer = StandardScaler()

encoder = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_var),
        ('num', num_transformer, num_var)
    ],
)

In [41]:
# Pipeline
pipeline = Pipeline([
    ('enc', encoder),
    ('model', DummyRegressor(strategy="mean")),
])

In [42]:
# Jeu de données
df = df_tel.reset_index()
df = df[(df['date'] > '2019-12-31') & (df['date'] > '2023-01-01')].drop(["id", "date"], axis=1)

y = df['nb_recus']
X = df

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, train_size=0.70, random_state=42)

In [43]:
# Entrainement
pipeline.fit(X_train, y_train)

Pipeline(steps=[('enc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['famille', 'jour', 'mois',
                                                   'jour_int']),
                                                 ('num', StandardScaler(),
                                                  ['nb_recus', 'vacances'])])),
                ('model', DummyRegressor())])

In [44]:
pipeline.score(X_test, y_test)

-0.0015515902078580002